In [1]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()

In [2]:
# Khởi tạo SparkSession
from pyspark.sql import SparkSession
spark.builder.appName("MongoDB to Spark and PostgreSQL")
spark.builder.config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.2,org.postgresql:postgresql:42.7.4")
spark.builder.config("spark.mongodb.input.uri", "mongodb://localhost:27017/dbjobcrawler")
spark.builder.getOrCreate()

In [3]:
#doc du lieu tu mongo
df=spark.read.format("mongo").option("collection","tbljob").load()
# Hiển thị schema 
df.printSchema()
#hien du lieu
df.show()

Py4JJavaError: An error occurred while calling o32.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: mongo. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:172)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Unknown Source)
	at java.lang.reflect.Method.invoke(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(M

In [49]:
#Xóa các cột không cần thiết
df = df.drop("_id","Chi_tiet_link")
# Hiển thị dữ liệu sau khi xóa cột
df.show()


+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|            Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|   100 m²|       10 triệu|121237|Thanh lý nhà Lý T...|05/10/2024|          NULL|   Cam Ranh|     2 phòng|        3 phòng|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|   200 m²|11 tỷ 090 triệu|121187|💎💎💎 HIẾM DÃY T...|02/10/2024|Sổ đỏ/ Sổ hồng|     Gò Vấp|    15 phòng|       15 phòng|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|    56 m²| 9 tỷ 950 triệu|121146|Bán nhà HXH hạ ch...|02/10/2024|Sổ đỏ/ Sổ hồng|  Hoàn Kiếm|     4 phòng|        5 ph

In [50]:
df.columns

['Diện_tích',
 'Giá',
 'Mã_tin',
 'Mô_tả',
 'Ngày_đăng',
 'Pháp_lý',
 'Quận',
 'Số_phòng_ngủ',
 'Số_phòng_toilet',
 'Thành_phố',
 'Tiêu_đề',
 'Đường']

In [51]:
from pyspark.sql import functions as F

def remove_special_characters_pyspark(df, text_column):
    # Xử lý biểu tượng emoji và ký tự đặc biệt
    emoji_pattern = "[" \
        "\U0001F600-\U0001F64F"  \
        "\U0001F300-\U0001F5FF"  \
        "\U0001F680-\U0001F6FF"  \
        "\U0001F1E0-\U0001F1FF"  \
        "\u2600-\u26FF"  \
        "\u2700-\u27BF"  \
        "]"
    
    # Loại bỏ emoji
    df = df.withColumn(text_column, F.regexp_replace(text_column, emoji_pattern, ""))
    
    # Loại bỏ các ký tự đặc biệt, chỉ giữ lại các ký tự tiếng Việt, chữ cái, số, khoảng trắng và dấu câu cơ bản
    allowed_chars = r'[^a-zA-Z0-9\s.,áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÍÌỈĨỊÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ]'
    
    df = df.withColumn(text_column, F.regexp_replace(text_column, allowed_chars, ""))
    
    return df
# Giả sử DataFrame có cột 'text' chứa dữ liệu văn bản
df=remove_special_characters_pyspark(df, 'Mô_tả')
df=remove_special_characters_pyspark(df, 'Tiêu_đề')


In [52]:
# kiểm tra lại
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Biểu thức regex cho emoji
emoji_pattern = "[" \
    "\U0001F600-\U0001F64F"  \
    "\U0001F300-\U0001F5FF"  \
    "\U0001F680-\U0001F6FF"  \
    "\U0001F1E0-\U0001F1FF"  \
    "\u2600-\u26FF"  \
    "\u2700-\u27BF"  \
    "]"

# Hàm Python kiểm tra xem chuỗi có chứa emoji không
def contains_emoji(text):
    if re.search(emoji_pattern, text):
        return True
    return False

# Đăng ký hàm UDF trong PySpark
contains_emoji_udf = udf(contains_emoji, BooleanType())


In [53]:
# Áp dụng UDF để kiểm tra emoji trong cột 'Mô_tả' và 'Tiêu_đề'
df_with_emoji = df.withColumn('has_emoji_in_Mo_ta', contains_emoji_udf(df['Mô_tả'])) \
                  .withColumn('has_emoji_in_Tieu_de', contains_emoji_udf(df['Tiêu_đề']))

# Lọc những hàng chứa emoji
emoji_rows = df_with_emoji.filter(F.col('has_emoji_in_Mo_ta') | F.col('has_emoji_in_Tieu_de'))

# Đếm số hàng chứa emoji
emoji_count = emoji_rows.count()
print(f"Số hàng chứa emoji: {emoji_count}")


Số hàng chứa emoji: 0


In [54]:
df.show()

+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|            Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|   100 m²|       10 triệu|121237|Thanh lý nhà Lý T...|05/10/2024|          NULL|   Cam Ranh|     2 phòng|        3 phòng|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|   200 m²|11 tỷ 090 triệu|121187| HIẾM DÃY TRỌ 1 X...|02/10/2024|Sổ đỏ/ Sổ hồng|     Gò Vấp|    15 phòng|       15 phòng|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|    56 m²| 9 tỷ 950 triệu|121146|Bán nhà HXH hạ ch...|02/10/2024|Sổ đỏ/ Sổ hồng|  Hoàn Kiếm|     4 phòng|        5

In [55]:
from pyspark.sql import functions as F

def remove_text_pyspark(df, text_column):
    # Loại bỏ tất cả ký tự không phải là chữ số
    df = df.withColumn(text_column, F.regexp_replace(text_column, r'[^\d]', ''))
    return df
df=remove_text_pyspark(df,'Diện_tích')
df=remove_text_pyspark(df,'Số_phòng_ngủ')
df=remove_text_pyspark(df,'Số_phòng_toilet')


In [56]:
from pyspark.sql import functions as F

# Lọc các hàng có giá trị không phải là số trong cột 'Diện_tích'
non_numeric_dien_tich = df.filter(F.col('Diện_tích').rlike(r'[^\d]'))

# Lọc các hàng có giá trị không phải là số trong cột 'Số_phòng_ngủ'
non_numeric_so_phong_ngu = df.filter(F.col('Số_phòng_ngủ').rlike(r'[^\d]'))

# Lọc các hàng có giá trị không phải là số trong cột 'Số_phòng_toilet'
non_numeric_so_phong_toilet = df.filter(F.col('Số_phòng_toilet').rlike(r'[^\d]'))

# Đếm số lượng hàng chứa ký tự không phải số trong từng cột
non_numeric_dien_tich_count = non_numeric_dien_tich.count()
non_numeric_so_phong_ngu_count = non_numeric_so_phong_ngu.count()
non_numeric_so_phong_toilet_count = non_numeric_so_phong_toilet.count()

print(f"Số hàng có giá trị không phải số trong cột 'Diện_tích': {non_numeric_dien_tich_count}")
print(f"Số hàng có giá trị không phải số trong cột 'Số_phòng_ngủ': {non_numeric_so_phong_ngu_count}")
print(f"Số hàng có giá trị không phải số trong cột 'Số_phòng_toilet': {non_numeric_so_phong_toilet_count}")


Số hàng có giá trị không phải số trong cột 'Diện_tích': 0
Số hàng có giá trị không phải số trong cột 'Số_phòng_ngủ': 0
Số hàng có giá trị không phải số trong cột 'Số_phòng_toilet': 0


In [57]:
df.show()

+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|            Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+---------------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|      100|       10 triệu|121237|Thanh lý nhà Lý T...|05/10/2024|          NULL|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|      200|11 tỷ 090 triệu|121187| HIẾM DÃY TRỌ 1 X...|02/10/2024|Sổ đỏ/ Sổ hồng|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|       56| 9 tỷ 950 triệu|121146|Bán nhà HXH hạ ch...|02/10/2024|Sổ đỏ/ Sổ hồng|  Hoàn Kiếm|           4|         

In [58]:
# Hiển thị schema sau khi xử lý
df.printSchema()

root
 |-- Diện_tích: string (nullable = true)
 |-- Giá: string (nullable = true)
 |-- Mã_tin: string (nullable = true)
 |-- Mô_tả: string (nullable = true)
 |-- Ngày_đăng: string (nullable = true)
 |-- Pháp_lý: string (nullable = true)
 |-- Quận: string (nullable = true)
 |-- Số_phòng_ngủ: string (nullable = true)
 |-- Số_phòng_toilet: string (nullable = true)
 |-- Thành_phố: string (nullable = true)
 |-- Tiêu_đề: string (nullable = true)
 |-- Đường: string (nullable = true)



In [59]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType

# Định nghĩa hàm xử lý giá
def xu_ly_gia(price_str):
    if price_str is None:  # Kiểm tra giá trị None
        return 0  # Hoặc bạn có thể trả về một giá trị khác nếu cần

    units = {
        'tỷ': 1000000000,
        'triệu': 1000000,
        'nghìn': 1000,
    }

    total = 0
    current_value = 0
    parts = price_str.split()
    
    for part in parts:
        if part in units:
            total += current_value * units[part]
            current_value = 0
        else:
            try:
                current_value = int(part)
            except ValueError:
                current_value = 0  # Trường hợp không thể chuyển đổi
    total += current_value
    return total

# Chuyển hàm Python thành UDF trong PySpark
xu_ly_gia_udf = F.udf(xu_ly_gia, LongType())  # Sử dụng LongType để tương thích

# Áp dụng UDF lên cột 'Giá' trong DataFrame và thay đổi cột 'Giá'
df = df.withColumn('Giá', xu_ly_gia_udf(F.col('Giá')))

# Hiển thị DataFrame sau khi chuyển đổi
df.show()


+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|      100|   10000000|121237|Thanh lý nhà Lý T...|05/10/2024|          NULL|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|      200|11090000000|121187| HIẾM DÃY TRỌ 1 X...|02/10/2024|Sổ đỏ/ Sổ hồng|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|       56| 9950000000|121146|Bán nhà HXH hạ ch...|02/10/2024|Sổ đỏ/ Sổ hồng|  Hoàn Kiếm|           4|              5|     Hà Nội|Bán n

In [60]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import IntegerType, FloatType, LongType

# Chuyển đổi kiểu dữ liệu
df = df.withColumn("Diện_tích", col("Diện_tích").cast(FloatType())) \
       .withColumn("Giá", col("Giá").cast(LongType()))\
       .withColumn("Số_phòng_ngủ", col("Số_phòng_ngủ").cast(IntegerType())) \
       .withColumn("Số_phòng_toilet", col("Số_phòng_toilet").cast(IntegerType())) \
       .withColumn("Ngày_đăng", to_date(col("Ngày_đăng"), "dd/MM/yyyy"))

# Hiển thị schema sau khi xử lý
df.printSchema()

# Hiển thị dữ liệu sau khi xử lý
df.show()


root
 |-- Diện_tích: float (nullable = true)
 |-- Giá: long (nullable = true)
 |-- Mã_tin: string (nullable = true)
 |-- Mô_tả: string (nullable = true)
 |-- Ngày_đăng: date (nullable = true)
 |-- Pháp_lý: string (nullable = true)
 |-- Quận: string (nullable = true)
 |-- Số_phòng_ngủ: integer (nullable = true)
 |-- Số_phòng_toilet: integer (nullable = true)
 |-- Thành_phố: string (nullable = true)
 |-- Tiêu_đề: string (nullable = true)
 |-- Đường: string (nullable = true)

+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|       Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+--------------+-----------+------------+---------------+-----------+--------------------+--------------------+
|  

In [61]:
# Lấy các giá trị khác nhau trong cột 'Pháp_lý'
distinct_phap_ly_values = df.select('Pháp_lý').distinct()

# Hiển thị các giá trị khác nhau
distinct_phap_ly_values.show(truncate=False)


+----------------+
|Pháp_lý         |
+----------------+
|Hợp đồng mua bán|
|Sổ đỏ/ Sổ hồng  |
|NULL            |
+----------------+



In [62]:
from pyspark.sql import functions as F

# Tích hợp phần trước ký tự '/' trong cột 'Pháp_lý'
df = df.withColumn('Pháp_lý', F.split(F.col('Pháp_lý'), '/').getItem(0))


In [63]:
# kiểm tra lại 
# Lấy các giá trị khác nhau trong cột 'Pháp_lý'
distinct_phap_ly_values = df.select('Pháp_lý').distinct()

# Hiển thị các giá trị khác nhau
distinct_phap_ly_values.show(truncate=False)


+----------------+
|Pháp_lý         |
+----------------+
|Hợp đồng mua bán|
|Sổ đỏ           |
|NULL            |
+----------------+



In [64]:
# Hiển thị DataFrame sau khi thực hiện
df.show()

+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|    100.0|   10000000|121237|Thanh lý nhà Lý T...|2024-10-05|   NULL|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|    200.0|11090000000|121187| HIẾM DÃY TRỌ 1 X...|2024-10-02|  Sổ đỏ|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|     56.0| 9950000000|121146|Bán nhà HXH hạ ch...|2024-10-02|  Sổ đỏ|  Hoàn Kiếm|           4|              5|     Hà Nội|Bán nhà HXH hạ ch...|       Trần Hưng Đạo|
|   

In [65]:
df.columns

['Diện_tích',
 'Giá',
 'Mã_tin',
 'Mô_tả',
 'Ngày_đăng',
 'Pháp_lý',
 'Quận',
 'Số_phòng_ngủ',
 'Số_phòng_toilet',
 'Thành_phố',
 'Tiêu_đề',
 'Đường']

In [66]:
# Tạo danh sách các cột kiểm tra giá trị null để dễ dàng xử lý
columns_to_check = ['Diện_tích', 'Giá', 'Mã_tin', 'Mô_tả', 'Ngày_đăng', 'Pháp_lý', 'Quận', 'Số_phòng_ngủ', 'Số_phòng_toilet', 'Thành_phố', 'Tiêu_đề', 'Đường']

# Lặp qua từng cột và đếm số giá trị null
for col in columns_to_check:
    null_count = df.filter(F.col(col).isNull()).count()  # Đếm số giá trị null trong cột
    print(f"Số giá trị null trong cột '{col}': {null_count}")


Số giá trị null trong cột 'Diện_tích': 32
Số giá trị null trong cột 'Giá': 0
Số giá trị null trong cột 'Mã_tin': 0
Số giá trị null trong cột 'Mô_tả': 0
Số giá trị null trong cột 'Ngày_đăng': 0
Số giá trị null trong cột 'Pháp_lý': 10
Số giá trị null trong cột 'Quận': 0
Số giá trị null trong cột 'Số_phòng_ngủ': 138
Số giá trị null trong cột 'Số_phòng_toilet': 191
Số giá trị null trong cột 'Thành_phố': 0
Số giá trị null trong cột 'Tiêu_đề': 0
Số giá trị null trong cột 'Đường': 0


In [67]:
num_rows = df.count()
num_columns = len(df.columns)
print(f"Kích thước của DataFrame: {num_rows} hàng và {num_columns} cột")


Kích thước của DataFrame: 1008 hàng và 12 cột


In [68]:
# Xử Lý giá trị null cột Diện_tích
# Thay thế NULL bằng giá trị trung vị
median_area = df.approxQuantile("Diện_tích", [0.5], 0)[0]
df = df.fillna({"Diện_tích": median_area})

df.show()

+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|    100.0|   10000000|121237|Thanh lý nhà Lý T...|2024-10-05|   NULL|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|    200.0|11090000000|121187| HIẾM DÃY TRỌ 1 X...|2024-10-02|  Sổ đỏ|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|     56.0| 9950000000|121146|Bán nhà HXH hạ ch...|2024-10-02|  Sổ đỏ|  Hoàn Kiếm|           4|              5|     Hà Nội|Bán nhà HXH hạ ch...|       Trần Hưng Đạo|
|   

In [69]:
#xử lý giá trị null của Số_phòng_ngủ và Số_phòng_toilet bằng 0
df = df.fillna({"Số_phòng_ngủ": 0, "Số_phòng_toilet": 0})
df.show()


+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|    100.0|   10000000|121237|Thanh lý nhà Lý T...|2024-10-05|   NULL|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|    200.0|11090000000|121187| HIẾM DÃY TRỌ 1 X...|2024-10-02|  Sổ đỏ|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|     56.0| 9950000000|121146|Bán nhà HXH hạ ch...|2024-10-02|  Sổ đỏ|  Hoàn Kiếm|           4|              5|     Hà Nội|Bán nhà HXH hạ ch...|       Trần Hưng Đạo|
|   

In [70]:
#Thay thế giá trị null của Pháp_lý thành "Unknown"
df = df.fillna({"Pháp_lý": "Unknown"})
df.show()


+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|    100.0|   10000000|121237|Thanh lý nhà Lý T...|2024-10-05|Unknown|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|    200.0|11090000000|121187| HIẾM DÃY TRỌ 1 X...|2024-10-02|  Sổ đỏ|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|     56.0| 9950000000|121146|Bán nhà HXH hạ ch...|2024-10-02|  Sổ đỏ|  Hoàn Kiếm|           4|              5|     Hà Nội|Bán nhà HXH hạ ch...|       Trần Hưng Đạo|
|   

In [71]:
#kiểm tra lại 
# Tạo danh sách các cột kiểm tra giá trị null để dễ dàng xử lý
columns_to_check = ['Diện_tích', 'Giá', 'Mã_tin', 'Mô_tả', 'Ngày_đăng', 'Pháp_lý', 'Quận', 'Số_phòng_ngủ', 'Số_phòng_toilet', 'Thành_phố', 'Tiêu_đề', 'Đường']

# Lặp qua từng cột và đếm số giá trị null
for col in columns_to_check:
    null_count = df.filter(F.col(col).isNull()).count()  # Đếm số giá trị null trong cột
    print(f"Số giá trị null trong cột '{col}': {null_count}")


Số giá trị null trong cột 'Diện_tích': 0
Số giá trị null trong cột 'Giá': 0
Số giá trị null trong cột 'Mã_tin': 0
Số giá trị null trong cột 'Mô_tả': 0
Số giá trị null trong cột 'Ngày_đăng': 0
Số giá trị null trong cột 'Pháp_lý': 0
Số giá trị null trong cột 'Quận': 0
Số giá trị null trong cột 'Số_phòng_ngủ': 0
Số giá trị null trong cột 'Số_phòng_toilet': 0
Số giá trị null trong cột 'Thành_phố': 0
Số giá trị null trong cột 'Tiêu_đề': 0
Số giá trị null trong cột 'Đường': 0


In [72]:
# Đếm số giá trị bằng 0 trong cột Giá
zero_count = df.filter(F.col("Giá") == 0).count()
# In ra số lượng giá trị bằng 0
print(f"Số giá trị bằng 0 trong cột 'Giá': {zero_count}")


Số giá trị bằng 0 trong cột 'Giá': 5


In [73]:
# nhận thấy có 5 giá trị của cột Giá=0 nên xóa đi
# Xóa các dòng có giá trị bằng 0 trong cột 'Giá'
df = df.filter(F.col("Giá") != 0)

# Hiển thị DataFrame sau khi xóa
df.show()


+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|Diện_tích|        Giá|Mã_tin|               Mô_tả| Ngày_đăng|Pháp_lý|       Quận|Số_phòng_ngủ|Số_phòng_toilet|  Thành_phố|             Tiêu_đề|               Đường|
+---------+-----------+------+--------------------+----------+-------+-----------+------------+---------------+-----------+--------------------+--------------------+
|    100.0|   10000000|121237|Thanh lý nhà Lý T...|2024-10-05|Unknown|   Cam Ranh|           2|              3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|    200.0|11090000000|121187| HIẾM DÃY TRỌ 1 X...|2024-10-02|  Sổ đỏ|     Gò Vấp|          15|             15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
|     56.0| 9950000000|121146|Bán nhà HXH hạ ch...|2024-10-02|  Sổ đỏ|  Hoàn Kiếm|           4|              5|     Hà Nội|Bán nhà HXH hạ ch...|       Trần Hưng Đạo|
|   

In [74]:
#xem lại kích thước
num_rows = df.count()
num_columns = len(df.columns)
print(f"Kích thước của DataFrame: {num_rows} hàng và {num_columns} cột")


Kích thước của DataFrame: 1003 hàng và 12 cột


In [75]:
# đổi tên cột
df = df \
    .withColumnRenamed("Diện_tích", "Area") \
    .withColumnRenamed("Mã_tin", "message_code") \
    .withColumnRenamed("Giá", "Price") \
    .withColumnRenamed("Mô_tả", "Description") \
    .withColumnRenamed("Ngày_đăng", "Posted_Date") \
    .withColumnRenamed("Pháp_lý", "Legal_Status") \
    .withColumnRenamed("Quận", "District") \
    .withColumnRenamed("Số_phòng_ngủ", "Bedrooms") \
    .withColumnRenamed("Số_phòng_toilet", "Bathrooms") \
    .withColumnRenamed("Thành_phố", "City") \
    .withColumnRenamed("Tiêu_đề", "Title") \
    .withColumnRenamed("Đường", "Street_Name")

# Hiển thị dữ liệu sau khi đổi tên cột
df.show()


+-----+-----------+------------+--------------------+-----------+------------+-----------+--------+---------+-----------+--------------------+--------------------+
| Area|      Price|message_code|         Description|Posted_Date|Legal_Status|   District|Bedrooms|Bathrooms|       City|               Title|         Street_Name|
+-----+-----------+------------+--------------------+-----------+------------+-----------+--------+---------+-----------+--------------------+--------------------+
|100.0|   10000000|      121237|Thanh lý nhà Lý T...| 2024-10-05|     Unknown|   Cam Ranh|       2|        3|  Khánh Hòa|Thanh lý nhà Lý T...|Thanh lý nhà Lý T...|
|200.0|11090000000|      121187| HIẾM DÃY TRỌ 1 X...| 2024-10-02|       Sổ đỏ|     Gò Vấp|      15|       15|Hồ Chí Minh|BÁN GẤP DÃY NHÀ T...|         Quang Trung|
| 56.0| 9950000000|      121146|Bán nhà HXH hạ ch...| 2024-10-02|       Sổ đỏ|  Hoàn Kiếm|       4|        5|     Hà Nội|Bán nhà HXH hạ ch...|       Trần Hưng Đạo|
| 75.0| 20000000

In [76]:
df.columns

['Area',
 'Price',
 'message_code',
 'Description',
 'Posted_Date',
 'Legal_Status',
 'District',
 'Bedrooms',
 'Bathrooms',
 'City',
 'Title',
 'Street_Name']

In [77]:
#Tạo các bản con
from pyspark.sql import functions as F
# 1. Tạo bảng House_Info: Thông tin về diện tích, giá, số phòng
house_info_df = df.select("Title","Area","Price", "Bedrooms", "Bathrooms") \
    .withColumn("House_ID", F.monotonically_increasing_id()) \
    .withColumnRenamed("Title", "House_Title") \
    .withColumnRenamed("Area", "House_Area") \
    .withColumnRenamed("Price", "House_Price") \
    .withColumnRenamed("Bedrooms", "House_Bedrooms") \
    .withColumnRenamed("Bathrooms", "House_Bathrooms")

# Hiển thị bảng House_Info
house_info_df.show(truncate=False)



+-----------------------------------------------------------------------------------------+----------+-----------+--------------+---------------+--------+
|House_Title                                                                              |House_Area|House_Price|House_Bedrooms|House_Bathrooms|House_ID|
+-----------------------------------------------------------------------------------------+----------+-----------+--------------+---------------+--------+
|Thanh lý nhà Lý Thái Tổ, P10, Q10  50m2 2 tỷ 5 1T1L  sổ riêng. HĐT 13tr tháng. HXH 6m    |100.0     |10000000   |2             |3              |0       |
|BÁN GẤP DÃY NHÀ TRỌ 200M2, QUANG TRUNG, 14 PHÒNG TRỌ P11 GÒ VẤP, GIÁ NHỈNH 11 ĐỒNG       |200.0     |11090000000|15            |15             |1       |
|Bán nhà HXH hạ chào hơn 4 tỷ, Trần Hưng Đạo, P1, Q5, 56m2, 4 tầng. 9.x tỷ                |56.0      |9950000000 |4             |5              |2       |
|bán gấp căn CCMN 75m 3 ngủ , 2 vệ sinh tại Giáp Nhất  Thanh Xuân Giá 

In [78]:
# 2. Tạo bảng Transaction: Chứa thông tin giao dịch
transaction_df = df.select("Title","Posted_Date", "Legal_Status") \
    .withColumn("Transaction_ID", F.monotonically_increasing_id()) \
    .withColumnRenamed("Title", "House_Title") \
    .withColumnRenamed("Posted_Date", "Transaction_Date") \
    .withColumnRenamed("Legal_Status", "Transaction_Status")  
# Hiển thị bảng Transaction
transaction_df.show(truncate=False)


+-----------------------------------------------------------------------------------------+----------------+------------------+--------------+
|House_Title                                                                              |Transaction_Date|Transaction_Status|Transaction_ID|
+-----------------------------------------------------------------------------------------+----------------+------------------+--------------+
|Thanh lý nhà Lý Thái Tổ, P10, Q10  50m2 2 tỷ 5 1T1L  sổ riêng. HĐT 13tr tháng. HXH 6m    |2024-10-05      |Unknown           |0             |
|BÁN GẤP DÃY NHÀ TRỌ 200M2, QUANG TRUNG, 14 PHÒNG TRỌ P11 GÒ VẤP, GIÁ NHỈNH 11 ĐỒNG       |2024-10-02      |Sổ đỏ             |1             |
|Bán nhà HXH hạ chào hơn 4 tỷ, Trần Hưng Đạo, P1, Q5, 56m2, 4 tầng. 9.x tỷ                |2024-10-02      |Sổ đỏ             |2             |
|bán gấp căn CCMN 75m 3 ngủ , 2 vệ sinh tại Giáp Nhất  Thanh Xuân Giá 2 tỷ                |2024-10-01      |Sổ đỏ             |3             |

In [79]:
# tạo khóa ngoại cho bảng transaction_df đến house_info_df
transaction_df = transaction_df.join(
    house_info_df.select("House_Title", "House_ID"),
    on="House_Title",
    how="inner"
)
transaction_df.show(truncate=False)

+------------------------------------------------------------------------------------------------+----------------+------------------+--------------+--------+
|House_Title                                                                                     |Transaction_Date|Transaction_Status|Transaction_ID|House_ID|
+------------------------------------------------------------------------------------------------+----------------+------------------+--------------+--------+
|Phân lô cạnh dự án Gold Mark City 136 Hồ Tùng Mậu 35m 4 tầng chỉ hơn 7 tỷ                       |2024-09-24      |Sổ đỏ             |411           |411     |
|Bán nhà Liêm Mạc  Vị trí đẹp  Nhà mới có thang máy  45m2  5.6 tỷ  5 tầng  cực kì tiềm năng.     |2024-09-21      |Sổ đỏ             |456           |456     |
|DƯƠNG QUẢNG HÀMCẦU GIẤY NHÀ XÂY MỚI 5 TẦNG x 31M2 6.86 TỶ NGÕ BA GÁC GẦN ÔTÔ                    |2024-09-24      |Sổ đỏ             |633           |633     |
|Bán nhà Đình Phong Phú , HXH ,2 tầng , 110m2,

In [80]:
# # 3. Tạo bảng Location: Chứa thông tin vị trí
# location_df = df.select("District", "City", "Street_Name") \
#     .withColumn("Location_ID", F.monotonically_increasing_id()) \
#     .withColumnRenamed("District", "District_Name") \
#     .withColumnRenamed("City", "City_Name") \
#     .withColumnRenamed("Street_Name", "House_Sreet") \

# # Hiển thị bảng Location
# location_df.show(truncate=False)


In [81]:
# # Tạo khóa ngoại cho bảng location_df đến transaction_df
# location_df = location_df.join(
#     transaction_df.select( "House_Date","Transaction_ID"),
#     on="House_Date", 
#     how="inner"
# )

# # Hiển thị bảng location_with_transaction
# location_df.show(truncate=False)

In [82]:
# Top 10 giá lớn nhất
from pyspark.sql import functions as F
from pyspark.sql.functions import col
top_10_Price= house_info_df.orderBy(col("House_Price").desc()).limit(10)

top_10_Price.show()

+--------------------+----------+------------+--------------+---------------+--------+
|         House_Title|House_Area| House_Price|House_Bedrooms|House_Bathrooms|House_ID|
+--------------------+----------+------------+--------------+---------------+--------+
|Bán Nhà Mặt Đường...|    2200.0|190000000000|             4|              4|     126|
|SIÊU PHẨM BIỆT TH...|    1000.0|150000000000|             0|              0|     179|
|Bán mặt tiền Quận...|     200.0|132000000000|            10|             10|     933|
|QUẬN 1Bán Nhà Mặt...|     170.0|115000000000|            15|             15|     923|
|HIẾMBán nhà mặt p...|     180.0|100000000000|            10|             10|     937|
| QUẬN 5Bán Nhà MT...|     160.0| 95000000000|             8|              8|     915|
|Cần Bán Biệt Thự ...|     369.0| 86000000000|            15|              8|     391|
|MẶT PHỐ NGHI TÀMT...|     126.0| 81000000000|             8|              0|     326|
|75TỶBán Tòa Nhà Đ...|     152.0| 750000000

In [83]:
# Top 10 giá nhỏ nhất
from pyspark.sql import functions as F
from pyspark.sql.functions import col
top_10_Price= house_info_df.orderBy(col("House_Price").asc()).limit(10)

top_10_Price.show()

+--------------------+----------+-----------+--------------+---------------+--------+
|         House_Title|House_Area|House_Price|House_Bedrooms|House_Bathrooms|House_ID|
+--------------------+----------+-----------+--------------+---------------+--------+
|Ban Nha Đươn Giên...|      68.0|       6170|             5|              3|     381|
|Bán đất Tiền phon...|     125.0|    3500000|             0|              0|     766|
|CHO THUÊ  Lý Chín...|      23.0|    8000000|             3|              2|     107|
|Thanh lý nhà Lý T...|     100.0|   10000000|             2|              3|       0|
|Chính chủ cho thu...|      50.0|   10500000|             2|              2|     735|
|Bán nhà XUÂN LA, ...|      50.0|   12500000|             0|              0|     377|
|NHÀ NGHĨA ĐÔ  PHÂ...|      35.0|   13000000|             0|              0|     376|
|Cho thuê nhà làm ...|      45.0|   15000000|             4|              4|      74|
|CHO THUÊ HOMESTAY...|      56.0|   20000000|         

In [84]:
# 4. Tạo bảng số lượng nhà bán theo thành phố
houses_by_city_df = df.groupBy("City") \
    .agg(F.count("Title").alias("Number_of_Houses")) \
    .withColumnRenamed("City", "City_Name") \
    .withColumn("City_ID", F.monotonically_increasing_id())  # Thêm City_ID làm khóa chính

# Lấy Top 10 thành phố có số lượng nhà bán cao nhất
top_10_cities_df = houses_by_city_df.orderBy(F.desc("Number_of_Houses")).limit(10)

# Hiển thị bảng Top 10 thành phố có số lượng nhà bán cao nhất
top_10_cities_df.show(truncate=False)

+--------------+----------------+-------+
|City_Name     |Number_of_Houses|City_ID|
+--------------+----------------+-------+
|Hồ Chí Minh   |541             |6      |
|Hà Nội        |402             |2      |
|Đồng Nai      |13              |11     |
|Bình Dương    |10              |8      |
|Đà Nẵng       |9               |1      |
|Long An       |5               |7      |
|Khánh Hòa     |4               |5      |
|Lâm Đồng      |4               |12     |
|Thừa Thiên Huế|4               |14     |
|Ninh Bình     |3               |9      |
+--------------+----------------+-------+



In [85]:
# 5. Tạo các bản con
from pyspark.sql import functions as F
# 1. Tạo bảng House_Info: Thông tin về diện tích, giá, số phòng
house_info_df = df.select("Area", "Price", "Bedrooms", "Bathrooms") \
    .withColumn("House_ID", F.monotonically_increasing_id()) \
    .withColumnRenamed("Area", "House_Area") \
    .withColumnRenamed("Price", "House_Price") \
    .withColumnRenamed("Bedrooms", "House_Bedrooms") \
    .withColumnRenamed("Bathrooms", "House_Bathrooms")

#tạo dataframe cho nhà có phòng ngủ thấp đến cao
top_hours_bedrooms_df=house_info_df\
    .orderBy(F.desc("House_Bedrooms"))\
    .limit(10)
    
#hiển thị
top_hours_bedrooms_df.show()



+----------+-----------+--------------+---------------+--------+
|House_Area|House_Price|House_Bedrooms|House_Bathrooms|House_ID|
+----------+-----------+--------------+---------------+--------+
|     225.0|31000000000|            45|             45|     457|
|     133.0|70000000000|            42|             46|     635|
|     162.0|60000000000|            35|             35|     213|
|     122.0|37000000000|            35|             35|     658|
|      81.0|18500000000|            27|             27|     500|
|     128.0|19500000000|            26|              0|     701|
|     100.0|22000000000|            24|             24|     877|
|     170.0|32000000000|            23|             23|     449|
|      99.0|35000000000|            22|             22|     806|
|     132.0|40000000000|            20|             20|     898|
+----------+-----------+--------------+---------------+--------+



In [86]:
# Kết nối tới PostgreSQL

jdbc_url = "jdbc:postgresql://localhost:5432/postgre_bds"

connection_properties = {
    "user": "postgres",
    "password": "1",
    "driver": "org.postgresql.Driver"
}

In [87]:
# Hàm ghi dữ liệu vào PostgreSQL

def write_to_postgres(df, table_name):
    try:
        df.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="overwrite",
            properties=connection_properties
        )
        print(f"Đã ghi thành công bảng {table_name} vào PostgreSQL!")
    except Exception as e:
        print(f"Lỗi khi ghi bảng {table_name} vào PostgreSQL: {e}")

In [88]:
# Lưu bảng vào PostgreSQL

write_to_postgres(house_info_df, "house_info_df")

write_to_postgres(transaction_df, "transaction_df")


Lỗi khi ghi bảng house_info_df vào PostgreSQL: An error occurred while calling o732.jdbc.
: org.postgresql.util.PSQLException: FATAL: database "postgre_bds" does not exist
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2733)
	at org.postgresql.core.v3.QueryExecutorImpl.readStartupMessages(QueryExecutorImpl.java:2845)
	at org.postgresql.core.v3.QueryExecutorImpl.<init>(QueryExecutorImpl.java:176)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:323)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:273)
	at org.postgresql.Driver.makeConnection(Driver.java:446)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.C

In [89]:
df.columns

['Area',
 'Price',
 'message_code',
 'Description',
 'Posted_Date',
 'Legal_Status',
 'District',
 'Bedrooms',
 'Bathrooms',
 'City',
 'Title',
 'Street_Name']

In [91]:
from pymongo import MongoClient
import psycopg2
import traceback


def create_table(pg_cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS courses (
        coursename TEXT PRIMARY KEY,
        lecturer TEXT,
        intro TEXT,
        describe TEXT,
        votenumber INTEGER,
        rating NUMERIC,
        newfee TEXT,
        oldfee TEXT,
        lessonnum INTEGER
    );
    """
    pg_cursor.execute(create_table_query)

# Kết nối tới MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['dbmycrawler']
mongo_collection = mongo_db['batdongsan_data']

# Kết nối tới PostgreSQL
pg_conn = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='1',
    host='localhost',
    port='5432'
)
pg_cursor = pg_conn.cursor()

try:
    # Tạo bảng nếu chưa tồn tại
    create_table(pg_cursor)

    # Truy xuất dữ liệu từ MongoDB
    documents = mongo_collection.find()

    # Chèn dữ liệu vào PostgreSQL
    for document in documents:
        # Kiểm tra nếu 'coursename' đã tồn tại
        pg_cursor.execute("SELECT COUNT(*) FROM courses WHERE coursename = %s", (document['Mã_tin'],))
        count = pg_cursor.fetchone()[0]
        
        if count == 0:  # Nếu không tồn tại, chèn dữ liệu
            pg_cursor.execute(
                """
                INSERT INTO courses (coursename, lecturer, intro, describe, votenumber, rating, newfee, oldfee, lessonnum)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                """,
                (
                    document['Mã_tin'],
                    document['Tiêu_đề'],
                    document['Đường'],
                    document['Quận'],
                    document['Thành_phố'],
                    document['Diện_tích'],
                    document['Giá'],
                    document['Pháp_lý'],
                    document['Số_phòng_ngủ']
                )
            )
        else:
            print(f"'{document['coursename']}' đã tồn tại trong cơ sở dữ liệu.")

    # Commit các thay đổi
    pg_conn.commit()

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")
    traceback.print_exc()
finally:
    # Đóng kết nối
    pg_cursor.close()
    pg_conn.close()
    mongo_client.close()

Đã xảy ra lỗi: 'coursename'


Traceback (most recent call last):
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_23516\2994431927.py", line 47, in <module>
    pg_cursor.execute("SELECT COUNT(*) FROM courses WHERE coursename = %s", (document['coursename'],))
KeyError: 'coursename'
